### Extract


In [ ]:
import requests
import json
import os
import time
from urllib.parse import quote


def get_next_page(response):
    """Extracts the next page URL from the API response."""
    return response.get("paging", {}).get("next")


def fetch_data(api_url):
    """Fetches paginated data from the API."""
    all_data = []
    
    while api_url:
        try:
            response = requests.get(api_url)
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
            
            json_data = response.json()
            data = json_data.get("data", [])
            all_data.extend(data)
            
            print(f"Extracted {len(data)} records. Total: {len(all_data)}")

            api_url = get_next_page(json_data)
            time.sleep(2)  # Respect API rate limits

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break

    return all_data


def extract_data(bylines, output_dir, access_token, year, api_version="v21.0", country="CZ", language="cs"):
    """
    Extracts and saves ad data from the Facebook Ads API for a list of bylines.
    
    Parameters:
        bylines (list): List of bylines to query.
        output_dir (str): Directory to save the JSON files.
        access_token (str): Facebook API access token.
        year (int): Year for filtering ads.
        api_version (str): API version (default: v21.0).
        country (str): Country code for filtering ads (default: CZ).
        language (str): Language code for filtering ads (default: cs).
    """
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

    fields = ",".join([
        "id", "ad_snapshot_url", "ad_creation_time", "ad_creative_bodies", "ad_creative_link_captions",
        "ad_creative_link_descriptions", "ad_creative_link_titles", "ad_delivery_start_time", 
        "ad_delivery_stop_time", "bylines", "currency", "delivery_by_region", "demographic_distribution", 
        "estimated_audience_size", "impressions", "languages", "page_id", "page_name", "publisher_platforms", 
        "spend", "target_locations", "target_gender", "target_ages", "eu_total_reach", "beneficiary_payers", 
        "age_country_gender_reach_breakdown"
    ])
    
    for byline in bylines:
        print(f"Extracting Data for: {byline}")
        encoded_byline = quote(f'["{byline}"]')

        api_url = (
            f"https://graph.facebook.com/{api_version}/ads_archive?"
            f"bylines={encoded_byline}&ad_type=POLITICAL_AND_ISSUE_ADS"
            f"&ad_reached_countries=['{country}']&access_token={access_token}"
            f"&unmask_removed_content=true&fields={fields}&limit=199"
            f"&search_terms=''&languages=['{language}']"
            f"&ad_delivery_date_min={year}-01-01"
            f"&ad_delivery_date_max={year+1}-01-01"
        )
        
        extracted_data = fetch_data(api_url)
        time.sleep(5)  # Delay between different bylines

        if extracted_data:
            filename = os.path.join(output_dir, f"data_{byline}.json")
            with open(filename, "w", encoding="utf-8") as json_file:
                json.dump(extracted_data, json_file, indent=4, ensure_ascii=False)
            print(f"Saved data to {filename}")
        else:
            print(f"No data extracted for {byline}.")


# Parameters
year = 2023
base_dir = os.getcwd()
output_dir = os.path.join(base_dir, f"bylines_ads/{year}")
# bylines = ["SH media, spol. s r.o."]
bylines = ['Česká pirátská strana', 'EUROPEUM Institute for European Policy', 'FTV Prima', 'ANO', 'Naše zdravotnictví', 'Svoboda a přímá demokracie (SPD)', 'ČSOB', 'ODS', 'Lékaři bez hranic - Médecins Sans Frontières in Czech Republic, o. p. s.', 'Aliance pro budoucnost', 'STAN', 'Starostové a nezávislí • STAN', 'PRAHA SOBĚ', 'Člověk v tísni, o.p.s.', 'SH media, spol. s r.o.', 'Komunistická strana Čech a Moravy', 'Zdeněk Hraba - senátor', 'Ondrej Prokop', 'Ministerstvo práce a sociálních věcí', 'Tomáš Zdechovský', 'Milion Chvilek, z. s.', 'EU Social', 'EU Justice and Consumers', 'ODS - Občanská demokratická strana', 'Sociální demokracie', 'Amnesty International ČR', 'Občanská demokratická strana', 'Tipsport', 'Karel Janeček', 'Martin Kuba', 'Starostové a nezávislí', 'Svoboda zvířat Plzeň, z.s.', 'CARE Česká republika', 'Replastuj.cz', 'ANO 2011', 'Reportér magazín', 'Andrej Babiš', 'Člověk v tísni o.p.s.', 'DFMG', 'TOP 09', 'SEN 21', 'Kupředu do minulosti s.r.o.', 'Ministerstvo pro místní rozvoj ČR', 'Svoboda a přímá demokracie', 'Greenpeace Česká republika', 'STAROSTOVÉ A NEZÁVISLÍ', 'KDU-ČSL', 'Česká pirátská strana - Praha', 'XTV', 'Hnutí DUHA - Přátelé Země Česká republika', 'Zelení - Strana zelených', 'PŘÍSAHA - občanské hnutí Roberta Šlachty', 'Český rozhlas', 'CZECH NEWS CENTER a. s.', 'Oldřich Hájek', 'Transparency International ČR']

access_token = "EAALnc8im5MUBO9FtPZCD8AcvH8w0CfpLw4BTgWtLI0Ivs7d71FI5BMirPdtx3ejqC4l8OUu8foPYjEEgsGtDZB3nyWeS2SSq2OGHYEo3yJnrawuZC6q5bz2BdQj7NSB4kYVQiYNVLFSofwgjtAICABUIMGpK6F1GlMo9t4uYS29XszNgHnC4tkYtLn1"  # Use environment variable

if not access_token:
    raise ValueError("Access token is missing. Set the 'FB_ACCESS_TOKEN' environment variable.")

# Execute extraction
extract_data(bylines, output_dir, access_token, year)


Extracting Data for: SH media, spol. s r.o.
Extracted 199 records. Total: 199
Extracted 30 records. Total: 229
Extracted 0 records. Total: 229
Saved data to c:\Users\jirip\Documents\Developer\python\political_ads\bylines_ads/2023\data_SH media, spol. s r.o..json


### Process


In [ ]:
import os
import json
import re


def replace_id_with_underscore_id(json_data):
    """Recursively replaces 'id' with '_id' in JSON data."""
    if isinstance(json_data, dict):
        if "id" in json_data:
            json_data["_id"] = json_data.pop("id")
        for value in json_data.values():
            replace_id_with_underscore_id(value)
    elif isinstance(json_data, list):
        for item in json_data:
            replace_id_with_underscore_id(item)


def process_json_files(folder_path, process_function):
    """Applies a transformation function to all JSON files in a given folder."""
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as json_file:
                data = json.load(json_file)

            processed_data = process_function(data)

            with open(file_path, "w", encoding="utf-8") as json_file:
                json.dump(processed_data, json_file, indent=4, ensure_ascii=False)


def append_json_files(folder_path):
    """Combines all JSON files in a directory into a single list."""
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as json_file:
                all_data.extend(json.load(json_file))
    return all_data


def remove_emojis(text):
    """Removes emojis and symbols from a given text."""
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs
        "\U0001F680-\U0001F6FF"  # Transport and Map Symbols
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002600-\U000026FF"  # Miscellaneous Symbols
        "\U00000200-\U00002BFF"  # Additional symbols
        "\U0001F004"             # Mahjong tiles
        "\U0001F0CF"             # Playing cards
        "\n"                     # Line break
        "]",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub("", text)


def clean_ad_creative_bodies(data):
    """Removes emojis from 'ad_creative_bodies' field in JSON data."""
    for item in data:
        if "ad_creative_bodies" in item:
            if isinstance(item["ad_creative_bodies"], str):
                item["ad_creative_bodies"] = remove_emojis(item["ad_creative_bodies"])
            elif isinstance(item["ad_creative_bodies"], list):
                item["ad_creative_bodies"] = [remove_emojis(body) for body in item["ad_creative_bodies"]]
    return data


# Process all years
for year in range(2022, 2026):
    base_dir = os.getcwd()
    output_dir = os.path.join(base_dir, f"bylines_ads/{year}")

    if not os.path.exists(output_dir):
        print(f"Skipping {year} - No data found.")
        continue

    print(f"Processing data for year {year}...")

    # Step 1: Replace 'id' with '_id' in all JSON files
    process_json_files(output_dir, replace_id_with_underscore_id)
    print(f"Replaced 'id' with '_id' in {year} data.")

    # Step 2: Append all JSON data into a single file
    appended_data = append_json_files(output_dir)
    all_data_file = os.path.join(output_dir, "data_all.json")
    with open(all_data_file, "w", encoding="utf-8") as json_file:
        json.dump(appended_data, json_file, indent=4, ensure_ascii=False)
    print(f"Appended {len(appended_data)} records into {all_data_file}.")

    # Step 3: Remove emojis from 'ad_creative_bodies'
    cleaned_data = clean_ad_creative_bodies(appended_data)
    cleaned_data_file = os.path.join(output_dir, "data_all_cleaned.json")
    with open(cleaned_data_file, "w", encoding="utf-8") as json_file:
        json.dump(cleaned_data, json_file, indent=4, ensure_ascii=False)
    print(f"Cleaned 'ad_creative_bodies' and saved to {cleaned_data_file}.\n")

print("Processing complete.")


# Remove the processed files after completion
for year in range(2022, 2026):
    output_dir = os.path.join(base_dir, f"bylines_ads\\{year}")
    file_path = os.path.join(output_dir, "data_all.json")
    
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Removed file: {file_path}")
    else:
        print(f"File not found, skipping: {file_path}")


### Push to mongo


In [ ]:
from pymongo import MongoClient
import json
import os
from tqdm import tqdm

client = MongoClient('mongodb://localhost:27017')
db = client["spending_db"]
collection = db["spending"]

for year in range(2022, 2026):
    base_dir = os.getcwd()
    output_dir = os.path.join(base_dir, f"bylines_ads\\{year}")
    file_path = os.path.join(output_dir, "data_all_cleaned.json")

    with open(file_path, "r", encoding="utf-8") as file:
        json_data = json.load(file)

    new_count = 0
    updated_count = 0

    print(f"\nProcessing year {year}...")
    for document in tqdm(json_data, desc="Inserting documents", unit="doc"):
        result = collection.replace_one(
            {"_id": document["_id"]},
            document,
            upsert=True
        )
        if result.upserted_id is not None:
            new_count += 1
        elif result.modified_count == 1:
            updated_count += 1

    print(f"Year {year} Results:")
    print(f"New documents inserted: {new_count}")
    print(f"Existing documents updated: {updated_count}\n")

client.close()

# Remove the processed files after completion
for year in range(2022, 2026):
    output_dir = os.path.join(base_dir, f"bylines_ads\\{year}")
    file_path = os.path.join(output_dir, "data_all_cleaned.json")
    
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Removed file: {file_path}")
    else:
        print(f"File not found, skipping: {file_path}")

In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')
db = client["spending_db"]
collection = db["spending"]

distinct_page_ids = collection.distinct("page_id")
# print(f"Distinct page_ids: {distinct_page_ids}")
print(f"Total distinct page_ids: {len(distinct_page_ids)}")


pipeline = [
    {"$group": {"_id": "$page_id", "page_names": {"$addToSet": "$page_name"}}},
    {"$project": {"_id": 1, "page_names": 1, "count": {"$size": "$page_names"}}},
    {"$match": {"count": {"$gt": 1}}}
]

results = collection.aggregate(pipeline)

for result in results:
    print(f"Page ID: {result['_id']}, Page Names: {result['page_names']}")

Total distinct page_ids: 634
Page ID: 273251297864533, Page Names: ['Michal Vronský, kandidát na starostu Prahy 3', 'Michal Vronský, starosta Prahy 3']
Page ID: 228361180513562, Page Names: ['e15.cz', 'Deník E15', 'Deník e15']
Page ID: 307710452690539, Page Names: ['Starostové pro jižní Moravu', 'Starostové a nezávislí - Jihomoravský kraj']
Page ID: 106420985474553, Page Names: ['Václav Ranc do Senátu', 'Václav Ranc - politik']
Page ID: 1708126532760089, Page Names: ['Renata Chmelová, zastupitelka Prahy 10', 'Renata Chmelová, senátorka Prahy 10']
Page ID: 551976351670874, Page Names: ['Zdeněk Hraba • váš senátor', 'Zdeněk Hraba • senátor', 'Zdeněk Hraba']
Page ID: 100441749442313, Page Names: ['David Procházka, zastupitel hl. m. Prahy', 'David Procházka']
Page ID: 109207234551285, Page Names: ['Andrea Brzobohatá politička', 'Andrea Brzobohatá do Senátu']
Page ID: 104022902208605, Page Names: ['Lenka Jüngling, zastupitelka města Přerova', 'Lenka Jüngling', 'Lenka Jüngling, kandidátka na

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import requests
import os
from pymongo import MongoClient

def setup_mongo_connection(uri="mongodb://localhost:27017", db_name="spending_db", collection_name="spending"):
    """Establishes a connection to MongoDB and returns the collection."""
    client = MongoClient(uri)
    return client[db_name][collection_name]

def setup_chrome_driver():
    """Initializes and returns a Selenium Chrome WebDriver."""
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode for efficiency
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    return webdriver.Chrome(service=service, options=options)

def navigate_to_page(driver, page_id, delay=2):
    """Navigates to the Facebook Ads Library page for a given page ID."""
    url = f"https://www.facebook.com/ads/library/?active_status=all&ad_type=political_and_issue_ads&country=CZ&is_targeted_country=false&media_type=all&search_type=page&source=ad-report&view_all_page_id={page_id}"
    driver.get(url)
    time.sleep(delay)

def download_image(driver, page_id, output_dir="ad_data"):
    """Downloads the logo image for a given page ID and saves it locally."""
    os.makedirs(output_dir, exist_ok=True)
    try:
        div_element = driver.find_element(By.CSS_SELECTOR, "div.x9f619.x1n2onr6.x1ja2u2z")
        image_element = div_element.find_element(By.TAG_NAME, "img")
        image_src = image_element.get_attribute("src")
        img_data = requests.get(image_src, timeout=5).content
        image_path = os.path.join(output_dir, f"{page_id}_logo.png")
        with open(image_path, 'wb') as handler:
            handler.write(img_data)
        print(f"Downloaded image for Page ID {page_id}")
    except Exception as e:
        print(f"Error downloading image for {page_id}: {e}")

def main():
    collection = setup_mongo_connection()
    page_ids = collection.distinct("page_id")
    
    driver = setup_chrome_driver()
    
    try:
        for page_id in page_ids:
            print(f"Processing Page ID: {page_id}")
            navigate_to_page(driver, page_id)
            download_image(driver, page_id)
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        driver.quit()
        print("Driver closed.")

if __name__ == "__main__":
    main()


Processing Page ID: 100174253112291
Downloaded image for Page ID 100174253112291
Processing Page ID: 100191411972962
Downloaded image for Page ID 100191411972962
Processing Page ID: 100201752249169
Downloaded image for Page ID 100201752249169
Processing Page ID: 100299219400946
Downloaded image for Page ID 100299219400946
Processing Page ID: 100347228813654
Downloaded image for Page ID 100347228813654
Processing Page ID: 100441749442313
Downloaded image for Page ID 100441749442313
Processing Page ID: 100505211433010
Downloaded image for Page ID 100505211433010
Processing Page ID: 100537783337910
Downloaded image for Page ID 100537783337910
Processing Page ID: 1005891386269746
Downloaded image for Page ID 1005891386269746
Processing Page ID: 100590722629554
Downloaded image for Page ID 100590722629554
Processing Page ID: 100591058741403
Downloaded image for Page ID 100591058741403
Processing Page ID: 100599206002185
Downloaded image for Page ID 100599206002185
Processing Page ID: 100631